<h1>01 – Data Preparation<span class="tocSkip"></span></h1>

This notebook covers the methodology used for flood risk estimation across **100 Indian smart cities**.

\- **Risk** is assessed through three key components: **Hazard**, **Vulnerability** and **Exposure**, representing physical flood drivers, geographic sensitivity, and population exposure, respectively.<br>
\- Geospatial datasets from satellite and ground observations are aggregated and normalized to compute a unified **Flood Risk Index**, which serves as the **target** variable. The underlying parameters are retained as **features** for modelling.<br>

This notebook demonstrates the flood risk estimation for a specific urban city. The methodology is scalable across all 100 Indian smart cities and urban cities in general. The accuracy of the estimated flood risk is validated using historical records and reports.

<h1>Methodology<span class="tocSkip"></span></h1>
<img src="../docs/data_preparation_flowchart.jpg" alt="Data Preparation Flowchart" width="800">

<h1>Datasets<span class="tocSkip"></span></h1>

<sub>

| Component      | Dataset | Pixel Size / Resolution | Availability / Years | Frequency | Use |
|---------------|---------|-------------------------|----------------------|-----------|-----|
| **Vulnerability** | [JRC Global Surface Water v1.4](https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_4_GlobalSurfaceWater) | 30 meters | 1984 – 2022 | - | Detecting permanent water bodies |
|               | [NASA SRTM 30m DEM](https://developers.google.com/earth-engine/datasets/catalog/USGS_SRTMGL1_003) | 30 meters | 2000 | One-time | Elevation / DEM data |
|               | [Landsat 8 C2 T1](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1) | 30 meters | 2013 – present | 16-days | Land cover / Surface imagery |
| **Exposure**  | [GHSL Population P2023A](https://developers.google.com/earth-engine/datasets/catalog/JRC_GHSL_P2023A_GHS_POP) | 100 meters | 1975 – 2030 | 5-year | Population distribution |
| **Hazard**    | [OpenLandMap Soil Texture](https://developers.google.com/earth-engine/datasets/catalog/OpenLandMap_SOL_SOL_TEXTURE-CLASS_USDA-TT_M_v02) | 250 meters | 1950 – 2018 | - | Soil texture classification |
|               | [MODIS MCD12Q1 Land Cover](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MCD12Q1) | 500 meters | 2001 – present | Annual | Land cover classification |
|               | [CHIRPS Daily Precipitation](https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_DAILY) | 5566 meters | 1981 – present | Daily | Daily precipitation |
|               | [TerraClimate Monthly Climate](https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE) | 4638.3 meters | 1958 – present | Monthly | Determining Precipitation & Runoff score thresholds |

</sub>

# Setup & Configs

In [ ]:
import ee

# ee.Authenticate()
ee.Initialize(project='ee-539srijansiddharth')

In [ ]:
city = 'Indore'

# Hazard
land_use_year = 2013
rainfall_start_date = '2020-01-01'
rainfall_end_date   = '2020-12-31'

# Vulnerability
landsat_year = 2019

# Exposure
pop_year = 2020

In [ ]:
# City asset
city_roi = ee.FeatureCollection(f"projects/ee-539srijansiddharth/assets/Smart-Cities-India/{city}")

# India asset
india_roi = ee.FeatureCollection(f"projects/ee-539srijansiddharth/assets/India")

In [ ]:
# Plotting requirements
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import geemap
import plotly.graph_objects as go
from PIL import Image as PILImage
import io
import base64
import math
import json
from IPython.display import display
from IPython.display import Image

# convert shapefile geometry to GeoDataFrame
features = city_roi.getInfo()['features']
city_gdf = gpd.GeoDataFrame.from_features(features)

features = india_roi.getInfo()['features']
india_gdf = gpd.GeoDataFrame.from_features(features)

# bounds & extent
city_bounds = city_gdf.geometry.bounds.values[0]
city_extent = [city_bounds[0], city_bounds[2], city_bounds[1], city_bounds[3]]

india_bounds = india_gdf.geometry.bounds.values[0]
india_extent = [india_bounds[0], india_bounds[2], india_bounds[1], india_bounds[3]]

# compute scale for downscaling before plotting
import geopy.distance
target_pixels = 200

xmin, xmax, ymin, ymax = city_extent
width_m = geopy.distance.distance((ymin, xmin), (ymin, xmax)).m
height_m = geopy.distance.distance((ymin, xmin), (ymax, xmin)).m
scale_x = width_m / target_pixels
scale_y = height_m / target_pixels
scale = max(scale_x, scale_y)

# custom cmaps
from matplotlib.colors import LinearSegmentedColormap

rainbow_cmap = LinearSegmentedColormap.from_list('custom_scale', ['blue', 'cyan', 'green', 'yellow', 'red'])
gnylrd_cmap = LinearSegmentedColormap.from_list('custom_scale', ['green', 'yellow', 'red'])

# plotly configs
def create_encoded_image(np_array, cmap, key, custom_minmax):
    if key in custom_minmax:
        min_val, max_val = custom_minmax[key]
    else:
        min_val = np.nanmin(np_array)
        max_val = np.nanmax(np_array)

    if max_val - min_val == 0:
        array_scaled_for_cmap = np.zeros_like(np_array)
    else:
        array_scaled_for_cmap = (np_array - min_val) / (max_val - min_val)

    colored = cmap(array_scaled_for_cmap)
    alpha_channel = np.where(np.isnan(np_array), 0, 255).astype(np.uint8)
    rgb_array = (colored[:, :, :3] * 255).astype(np.uint8)
    rgba_array = np.dstack((rgb_array, alpha_channel))
    image = PILImage.fromarray(rgba_array)
    buffer = io.BytesIO()
    image.save(buffer, format='PNG')
    buffer.seek(0)
    encoded_image = "data:image/png;base64," + base64.b64encode(buffer.read()).decode()
    return encoded_image

def get_zoom_level(bounds, map_width_px, map_height_px):
    lon_min, lat_min, lon_max, lat_max = bounds
    WORLD_DIM = {'height': 512, 'width': 512}
    ZOOM_MAX = 22

    def lat_to_mercator(lat):
        lat_rad = math.radians(lat)
        return math.log(math.tan(lat_rad / 2 + math.pi / 4))

    def zoom(map_px, world_px, fraction):
        if fraction <= 0:
            fraction = 1e-10
        return math.floor(math.log(map_px / world_px / fraction) / math.log(2))

    lat_fraction = (lat_to_mercator(lat_max) - lat_to_mercator(lat_min)) / (2 * math.pi)
    lon_fraction = (lon_max - lon_min) / 360

    lat_fraction = max(lat_fraction, 1e-10)
    lon_fraction = max(lon_fraction, 1e-10)

    lat_zoom = zoom(map_height_px, WORLD_DIM['height'], lat_fraction)
    lon_zoom = zoom(map_width_px, WORLD_DIM['width'], lon_fraction)

    zoom_level = min(lat_zoom, lon_zoom, ZOOM_MAX)
    return max(0, zoom_level)

geojson = city_roi.getInfo()
features = geojson['features']

boundary_traces = []
lons_all, lats_all = [], []

for feature in features:
    geom = feature['geometry']
    for ring in geom['coordinates']:
        lats, lons = zip(*[(coord[1], coord[0]) for coord in ring])
        lats_all.extend(lats)
        lons_all.extend(lons)

        boundary_traces.append(
            go.Scattermapbox(
                lat=lats,
                lon=lons,
                mode='lines',
                line=dict(color='black', width=0.5),
                showlegend=False,
                hoverinfo='skip'
            )
        )

map_width, map_height = 460, 450

lon_min, lon_max = min(lons_all), max(lons_all)
lat_min, lat_max = min(lats_all), max(lats_all)
extent = [lon_min, lon_max, lat_min, lat_max]

center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2

zoom_level = get_zoom_level([lon_min, lat_min, lon_max, lat_max], map_width, map_height)

def create_layer_and_scatter(np_array, cmap, key, custom_minmax):
    encoded_image = create_encoded_image(np_array, cmap, key, custom_minmax=custom_minmax)
    layer = dict(
        sourcetype='image',
        source=encoded_image,
        coordinates=[
            [lon_min, lat_max],
            [lon_max, lat_max],
            [lon_max, lat_min],
            [lon_min, lat_min],
        ],
        opacity=1
    )

    nrows, ncols = np_array.shape
    lons = np.linspace(lon_min, lon_max, ncols)
    lats = np.linspace(lat_max, lat_min, nrows)

    lon_grid, lat_grid = np.meshgrid(lons, lats)
    flat_lons = lon_grid.flatten()
    flat_lats = lat_grid.flatten()
    flat_vals = np_array.flatten()

    valid_mask = ~np.isnan(flat_vals)
    flat_lons = flat_lons[valid_mask]
    flat_lats = flat_lats[valid_mask]
    flat_vals = flat_vals[valid_mask]

    hover_texts = [f"Value: {v:.2f}" for v in flat_vals]

    scatter = go.Scattermapbox(
        lat=flat_lats,
        lon=flat_lons,
        mode='markers',
        marker=dict(size=5, color='rgba(0,0,0,0)'),
        hoverinfo='text',
        hovertext=hover_texts,
        showlegend=False,
    )

    return layer, scatter


# Hazard

## Definition

In [ ]:
# Import datasets
soil_texture = ee.Image('OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02')
modis_landcover = ee.ImageCollection('MODIS/061/MCD12Q1')
chirps_daily = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')

In [ ]:
# Annual average runoff determination using AMC + SCN method
soil_class = soil_texture.select('b0').clip(city_roi).rename('soil')

soil_grp = soil_class.expression(
    "(b('soil') > 10) ? 4" +
    ": (b('soil') > 4) ? 3" +
    ": (b('soil') > 1) ? 2" +
    ": (b('soil') > 0) ? 1" +
    ": 0"
).rename('soil')

modis = modis_landcover.filter(ee.Filter.calendarRange(land_use_year, land_use_year, 'year'))

lulc = modis.select('LC_Type1').first().clip(city_roi).rename('lulc')
lulc_soil = lulc.addBands(soil_grp)

CN_whole = lulc_soil.expression(
        "(b('soil') == 1) and(b('lulc')==1) ? 35" +
        ": (b('soil') == 1) and(b('lulc')==2) ? 25" +
            ": (b('soil') == 1) and(b('lulc')==3) ? 45" +
            ": (b('soil') == 1) and(b('lulc')==4) ? 39" +
            ": (b('soil') == 1) and(b('lulc')==5) ? 45" +
            ": (b('soil') == 1) and(b('lulc')==6) ? 49" +
            ": (b('soil') == 1) and(b('lulc')==7) ? 68" +
            ": (b('soil') == 1) and(b('lulc')==8) ? 36" +
            ": (b('soil') == 1) and(b('lulc')==9) ? 45" +
            ": (b('soil') == 1) and(b('lulc')==10) ? 30" +
            ": (b('soil') == 1) and(b('lulc')==11) ? 95" +
            ": (b('soil') == 1) and(b('lulc')==12) ? 67" +
            ": (b('soil') == 1) and(b('lulc')==13) ? 72" +
            ": (b('soil') == 1) and(b('lulc')==14) ? 63" +
            ": (b('soil') == 1) and(b('lulc')==15) ? 100" +
            ": (b('soil') == 1) and(b('lulc')==16) ? 74" +
            ": (b('soil') == 1) and(b('lulc')==17) ? 100" +
            ": (b('soil') == 2) and(b('lulc')==1) ? 50" +
            ": (b('soil') == 2) and(b('lulc')==2) ? 55" +
            ": (b('soil') == 2) and(b('lulc')==3) ? 66" +
            ": (b('soil') == 2) and(b('lulc')==4) ? 61" +
            ": (b('soil') == 2) and(b('lulc')==5) ? 66" +
            ": (b('soil') == 2) and(b('lulc')==6) ? 69" +
            ": (b('soil') == 2) and(b('lulc')==7) ? 79" +
            ": (b('soil') == 2) and(b('lulc')==8) ? 60" +
            ": (b('soil') == 2) and(b('lulc')==9) ? 66" +
            ": (b('soil') == 2) and(b('lulc')==10) ? 58" +
            ": (b('soil') == 2) and(b('lulc')==11) ? 95" +
            ": (b('soil') == 2) and(b('lulc')==12) ? 78" +
            ": (b('soil') == 2) and(b('lulc')==13) ? 82" +
            ": (b('soil') == 2) and(b('lulc')==14) ? 75" +
            ": (b('soil') == 2) and(b('lulc')==15) ? 100" +
            ": (b('soil') == 2) and(b('lulc')==16) ? 84" +
            ": (b('soil') == 2) and(b('lulc')==17) ? 100" +
                ": (b('soil') == 3) and(b('lulc')==1) ? 73" +
                ": (b('soil') == 3) and(b('lulc')==2) ? 70" +
                ": (b('soil') == 3) and(b('lulc')==3) ? 77" +
                ": (b('soil') == 3) and(b('lulc')==4) ? 74" +
                ": (b('soil') == 3) and(b('lulc')==5) ? 77" +
                ": (b('soil') == 3) and(b('lulc')==6) ? 79" +
                ": (b('soil') == 3) and(b('lulc')==7) ? 86" +
                ": (b('soil') == 3) and(b('lulc')==8) ? 73" +
                ": (b('soil') == 3) and(b('lulc')==9) ? 77" +
                ": (b('soil') == 3) and(b('lulc')==10) ? 71" +
                ": (b('soil') == 3) and(b('lulc')==11) ? 95" +
                ": (b('soil') == 3) and(b('lulc')==12) ? 85" +
                ": (b('soil') == 3) and(b('lulc')==13) ? 87" +
                ": (b('soil') == 3) and(b('lulc')==14) ? 83" +
                ": (b('soil') == 3) and(b('lulc')==15) ? 100" +
                ": (b('soil') == 3) and(b('lulc')==16) ? 90" +
                ": (b('soil') == 3) and(b('lulc')==17) ? 100" +
                "  : (b('soil') == 4) and(b('lulc')==1) ? 79" +
                    ": (b('soil') == 4) and(b('lulc')==2) ? 77" +
                    ": (b('soil') == 4) and(b('lulc')==3) ? 83" +
                    ": (b('soil') == 4) and(b('lulc')==4) ? 80" +
                    ": (b('soil') == 4) and(b('lulc')==5) ? 83" +
                    ": (b('soil') == 4) and(b('lulc')==6) ? 89" +
                    ": (b('soil') == 4) and(b('lulc')==7) ? 89" +
                    ": (b('soil') == 4) and(b('lulc')==8) ? 79" +
                    ": (b('soil') == 4) and(b('lulc')==9) ? 83" +
                    ": (b('soil') == 4) and(b('lulc')==10) ? 78" +
                    ": (b('soil') == 4) and(b('lulc')==11) ? 95" +
                    ": (b('soil') == 4) and(b('lulc')==12) ? 89" +
                    ": (b('soil') == 4) and(b('lulc')==13) ? 89" +
                    ": (b('soil') == 4) and(b('lulc')==14) ? 87" +
                    ": (b('soil') == 4) and(b('lulc')==15) ? 100" +
                    ": (b('soil') == 4) and(b('lulc')==16) ? 92" +
                    ": (b('soil') == 4) and(b('lulc')==17) ? 100" +
                        ": (b('soil') == 0) ? 100" +
                        ": 0"
)

CN2 = CN_whole.clip(city_roi).rename('CN2')
CN1 = CN2.expression(
    'CN2 /(2.281 - (0.0128 * CN2))', {
        'CN2': CN2.select('CN2')
    }).rename('CN1')
CN3 = CN2.expression(
    'CN2 /(0.427+(0.00573*CN2))',{
    'CN2': CN2.select('CN2')
    }).rename('CN3')

S_image1 = CN1.expression(
    '(25400/CN1)-254', {
    'CN1': CN1.select('CN1')
}).rename('S_value1')

S_image2 = CN2.expression(
    '(25400/CN2)-254', {
    'CN2': CN2.select('CN2')
}).rename('S_value2')

S_image3 = CN3.expression(
    '(25400/CN3)-254', {
    'CN3': CN3.select('CN3')
}).rename('S_value3')

rainfall = chirps_daily.filter(ee.Filter.date(rainfall_start_date, rainfall_end_date))

listOfImages = rainfall.toList(rainfall.size())

def calculate_running_sum(img):
    img = ee.Image(img)
    index = listOfImages.indexOf(img)
    firstIndex = ee.Algorithms.If(index.lte(3), index, index.subtract(4))
    firstImage = ee.Image(listOfImages.get(firstIndex))
    secondIndex = ee.Algorithms.If(index.lte(3), index, index.subtract(3))
    secondImage = ee.Image(listOfImages.get(secondIndex))
    thirdIndex = ee.Algorithms.If(index.lte(3), index, index.subtract(2))
    thirdImage = ee.Image(listOfImages.get(thirdIndex))
    fourthIndex = ee.Algorithms.If(index.lte(3), index, index.subtract(1))
    fourthImage = ee.Image(listOfImages.get(fourthIndex))
    change = firstImage.add(secondImage).add(thirdImage).add(fourthImage).add(img) \
        .copyProperties(img, ["system:time_start"])
    return change

calculated_list = listOfImages.map(calculate_running_sum)

AMCcollection = ee.ImageCollection(calculated_list)

Join = ee.Join.inner()
FilterOnStartTime = ee.Filter.equals(
    leftField='system:time_start',
    rightField='system:time_start'
)
rain_AMC = Join.apply(rainfall, AMCcollection, FilterOnStartTime)

def MergeBands(aRow):
        return ee.Image.cat(aRow.get('primary'), aRow.get('secondary')).clip(city_roi)

merged = rain_AMC.map(MergeBands)
MergedRain_AMC = ee.ImageCollection(merged)

zeroImage = ee.Image(0)
def runoff_func(image):
    AMC = image.select('precipitation_1')
    ppt = image.select('precipitation')
    AMCreplaced = S_image2.where(AMC.lte(13), S_image1)
    AMCreplaced2 = AMCreplaced.where(AMC.gt(28), S_image3)
    s_value = AMCreplaced2.select('S_value2')

    Q2 = image.expression(
        '((ppt - (0.2 * S)) ** 2) / (ppt - (0.2 * S) + S)', {
            'ppt': ppt,
            'S': s_value
        })

    Q3 = Q2.where(ppt.lt(s_value.multiply(0.2)), zeroImage)
    return Q3.clip(city_roi).rename('runoff').copyProperties(image, ['system:time_start'])

runoff = MergedRain_AMC.map(runoff_func)

JoinedRR = Join.apply(rainfall, runoff, FilterOnStartTime)

RainfallRunoff1 = JoinedRR.map(MergeBands)
RainfallRunoff = ee.ImageCollection(RainfallRunoff1)

precipitationSum = RainfallRunoff.select('precipitation').sum()
runoffSum = RainfallRunoff.select('runoff').sum()

In [ ]:

hazard_data = {
    'Rainfall': precipitationSum,
    'Runoff': runoffSum
}

hazard_arrays = {}

for name, ee_image in hazard_data.items():
    arr = geemap.ee_to_numpy(
        ee_image.unmask(-9999),
        region=city_roi,
        scale=scale
    )
    arr = np.squeeze(arr)
    arr = arr.astype(float)
    arr[arr == -9999] = np.nan

    hazard_arrays[name] = arr

In [ ]:

pd.DataFrame({
    name: pd.Series(arr.ravel()[~np.isnan(arr.ravel())]).describe()
    for name, arr in hazard_arrays.items()
})

In [ ]:

keys = list(hazard_arrays.keys())

frames = []
for key in keys:
    layer, scatter = create_layer_and_scatter(hazard_arrays[key], gnylrd_cmap, key, {})
    frames.append(go.Frame(
        data=[scatter] + boundary_traces,
        name=key,
        layout=dict(mapbox_layers=[layer])
    ))

initial_layer, initial_scatter = create_layer_and_scatter(hazard_arrays[keys[0]], gnylrd_cmap, keys[0], {})

buttons = []
for key in keys:
    buttons.append({
        'label': key,
        'method': 'animate',
        'args': [[key], {
            'frame': {'duration': 500, 'redraw': True},
            'mode': 'immediate',
            'transition': {'duration': 300}
        }]
    })

fig = go.Figure(
    data=[initial_scatter] + boundary_traces,
    frames=frames,
    layout=go.Layout(
        width=map_width,
        height=map_height,
        margin=dict(l=0, r=0, t=0, b=0),
        mapbox=dict(
            style='open-street-map',
            center=dict(lat=center_lat, lon=center_lon),
            zoom=zoom_level,
            layers=[initial_layer]
        ),
        updatemenus=[{
            'buttons': buttons,
            'direction': 'down',
            'showactive': True,
            'x': 0.02,
            'y': 0.91,
            'xanchor': 'left',
            'yanchor': 'bottom'
        }]
    )
)

fig.show()
# png_bytes = fig.to_image(format="png")
# display(Image(png_bytes))

## Analysis

In [ ]:
terraclim = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")

### Precipitation

In [ ]:

years = list(range(1958, 2024 + 1))

arrays = []

for yr in years:
    pr_year = terraclim.filterDate(f"{yr}-01-01", f"{yr}-12-31") \
                       .select("pr") \
                       .sum() \
                       .clip(india_roi)

    np_array = geemap.ee_to_numpy(
        pr_year.unmask(-9999),
        region=india_roi,
        scale=5_000,
    )

    np_array = np.squeeze(np_array).astype(float)
    np_array[(np_array == -9999) | (np_array == 0)] = np.nan

    arrays.append(np_array)

pr_matrix = np.stack(arrays, axis=0)

In [ ]:

pd.DataFrame({
    yr: pd.Series(pr_matrix[i].ravel()[~np.isnan(pr_matrix[i].ravel())]).describe(percentiles=[0.10, 0.25, 0.5, 0.75, 0.9])
    for i, yr in enumerate(years)
}).agg(['median'], axis=1)

In [ ]:
median_pr = np.percentile(pr_matrix, 50, axis=0)

np.save("median_pr.npy", median_pr)
median_pr = np.load("median_pr.npy")

In [ ]:

plotly_colorscale = [
    [i / 255, f'rgb({int(r*255)}, {int(g*255)}, {int(b*255)})']
    for i, (r, g, b, _) in enumerate([rainbow_cmap(i / 255) for i in range(256)])
]

np_array = median_pr
min_val = np.nanmin(np_array)
max_val = np.nanmax(np_array)

nrows, ncols = np_array.shape
india_lon_min, india_lat_min, india_lon_max, india_lat_max = india_bounds

x = np.linspace(india_lon_min, india_lon_max, ncols)
y = np.linspace(india_lat_min, india_lat_max, nrows)

fig = go.Figure()

fig.add_trace(go.Heatmap(
    z=np_array[:, ::-1],
    x=x[::-1],
    y=y,
    colorscale=plotly_colorscale,
    zmin=min_val,
    zmax=max_val,
    showscale=False,
    hovertemplate='%{z}', name=''
))

fig.update_xaxes(
    scaleanchor="y",
    scaleratio=1,
    showticklabels=False,
    showgrid=False,
    zeroline=False,
    title=''
)
fig.update_yaxes(
    autorange="reversed",
    showticklabels=False,
    showgrid=False,
    zeroline=False,
    title=''
)
fig.update_layout(
    width=400,
    height=400,
    margin=dict(l=0, r=0, t=10, b=0),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.show()
# png_bytes = fig.to_image(format="png")
# display(Image(png_bytes))

In [ ]:

plotly_colorscale = [
    [i / 255, f'rgb({int(r*255)}, {int(g*255)}, {int(b*255)})']
    for i, (r, g, b, _) in enumerate([rainbow_cmap(i / 255) for i in range(256)])
]

np_array = median_pr
valid_values = np_array[~np.isnan(np_array)]

percentiles = np.percentile(valid_values, [0, 10, 25, 50, 75, 90, 100])

percentile_bins = np.digitize(np_array, percentiles, right=True) - 1
percentile_bins = np.clip(percentile_bins, 0, len(percentiles)-2)

percentile_bins = percentile_bins.astype(float)
percentile_bins[np.isnan(np_array)] = np.nan

colorscale_10 = [
    [i / (len(percentiles)-2), f'rgb({int(r*255)}, {int(g*255)}, {int(b*255)})']
    for i, (r, g, b, _) in enumerate([rainbow_cmap(i / (len(percentiles)-2)) for i in range(len(percentiles)-1)])
]

nrows, ncols = np_array.shape
india_lon_min, india_lat_min, india_lon_max, india_lat_max = india_bounds

x = np.linspace(india_lon_min, india_lon_max, ncols)
y = np.linspace(india_lat_min, india_lat_max, nrows)

fig = go.Figure()

fig.add_trace(go.Heatmap(
    z=percentile_bins[:, ::-1],
    x=x[::-1],
    y=y,
    colorscale=colorscale_10,
    zmin=0,
    zmax=len(percentiles)-2,
    showscale=False,
    customdata=np_array[:, ::-1],
    hovertemplate='Percentile bin: %{z}<br>Runoff: %{customdata}<extra></extra>',
    name=''
))

fig.update_xaxes(
    scaleanchor="y",
    scaleratio=1,
    showticklabels=False,
    showgrid=False,
    zeroline=False,
    title=''
)
fig.update_yaxes(
    autorange="reversed",
    showticklabels=False,
    showgrid=False,
    zeroline=False,
    title=''
)
fig.update_layout(
    width=400,
    height=400,
    margin=dict(l=0, r=0, t=10, b=0),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.show()
# png_bytes = fig.to_image(format="png")
# display(Image(png_bytes))

In [ ]:

np_array = median_pr
valid_values = np_array[~np.isnan(np_array)]

perc = [0, 10, 25, 50, 75, 90, 100]
percentiles = np.percentile(valid_values, perc)

percentile_bins = np.digitize(np_array, percentiles, right=True) - 1
percentile_bins = np.clip(percentile_bins, 0, len(percentiles)-2)

percentile_bins = percentile_bins.astype(float)

flat_bins = percentile_bins.flatten()
flat_values = np_array.flatten()

mask = ~np.isnan(flat_bins) & ~np.isnan(flat_values)
flat_bins = flat_bins[mask].astype(int)
flat_values = flat_values[mask]

df = pd.DataFrame({
    'percentile_bin': flat_bins,
    'rainfall': flat_values
})

summary = df.groupby('percentile_bin')['rainfall'].describe()
summary.index = perc[1:]
summary

### Runoff

In [ ]:

years =  list(range(1958, 2024 + 1))

arrays = []

for yr in years:
    ro_year = terraclim.filterDate(f"{yr}-01-01", f"{yr}-12-31") \
                       .select("ro") \
                       .sum() \
                       .clip(india_roi)

    np_array = geemap.ee_to_numpy(
        ro_year.unmask(-9999),
        region=india_roi,
        scale=5_000,
    )

    np_array = np.squeeze(np_array).astype(float)
    np_array[(np_array == -9999) | (np_array == 0)] = np.nan

    arrays.append(np_array)

ro_matrix = np.stack(arrays, axis=0)

In [ ]:

pd.DataFrame({
    yr: pd.Series(ro_matrix[i].ravel()[~np.isnan(pr_matrix[i].ravel())]).describe(percentiles=[0.10, 0.25, 0.5, 0.75, 0.9])
    for i, yr in enumerate(years)
})

In [ ]:
median_ro = np.percentile(ro_matrix, 50, axis=0)

np.save("median_ro.npy", median_ro)
median_ro = np.load("median_ro.npy")

In [ ]:

plotly_colorscale = [
    [i / 255, f'rgb({int(r*255)}, {int(g*255)}, {int(b*255)})']
    for i, (r, g, b, _) in enumerate([rainbow_cmap(i / 255) for i in range(256)])
]

np_array = median_ro
min_val = np.nanmin(np_array)
max_val = np.nanmax(np_array)

nrows, ncols = np_array.shape
india_lon_min, india_lat_min, india_lon_max, india_lat_max = india_bounds

x = np.linspace(india_lon_min, india_lon_max, ncols)
y = np.linspace(india_lat_min, india_lat_max, nrows)

fig = go.Figure()

fig.add_trace(go.Heatmap(
    z=np_array[:, ::-1],
    x=x[::-1],
    y=y,
    colorscale=plotly_colorscale,
    zmin=min_val,
    zmax=max_val,
    showscale=False,
    hovertemplate='%{z}', name=''
))

fig.update_xaxes(
    scaleanchor="y",
    scaleratio=1,
    showticklabels=False,
    showgrid=False,
    zeroline=False,
    title=''
)
fig.update_yaxes(
    autorange="reversed",
    showticklabels=False,
    showgrid=False,
    zeroline=False,
    title=''
)
fig.update_layout(
    width=400,
    height=400,
    margin=dict(l=0, r=0, t=10, b=0),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.show()
# png_bytes = fig.to_image(format="png")
# display(Image(png_bytes))

In [ ]:

plotly_colorscale = [
    [i / 255, f'rgb({int(r*255)}, {int(g*255)}, {int(b*255)})']
    for i, (r, g, b, _) in enumerate([rainbow_cmap(i / 255) for i in range(256)])
]

np_array = median_ro
valid_values = np_array[~np.isnan(np_array)]

percentiles = np.percentile(valid_values, [0, 10, 25, 50, 75, 90, 100])

percentile_bins = np.digitize(np_array, percentiles, right=True) - 1
percentile_bins = np.clip(percentile_bins, 0, len(percentiles)-2)

percentile_bins = percentile_bins.astype(float)
percentile_bins[np.isnan(np_array)] = np.nan

colorscale_10 = [
    [i / (len(percentiles)-2), f'rgb({int(r*255)}, {int(g*255)}, {int(b*255)})']
    for i, (r, g, b, _) in enumerate([rainbow_cmap(i / (len(percentiles)-2)) for i in range(len(percentiles)-1)])
]

nrows, ncols = np_array.shape
india_lon_min, india_lat_min, india_lon_max, india_lat_max = india_bounds

x = np.linspace(india_lon_min, india_lon_max, ncols)
y = np.linspace(india_lat_min, india_lat_max, nrows)

fig = go.Figure()

fig.add_trace(go.Heatmap(
    z=percentile_bins[:, ::-1],
    x=x[::-1],
    y=y,
    colorscale=colorscale_10,
    zmin=0,
    zmax=len(percentiles)-2,
    showscale=False,
    customdata=np_array[:, ::-1],
    hovertemplate='Percentile bin: %{z}<br>Runoff: %{customdata}<extra></extra>',
    name=''
))

fig.update_xaxes(
    scaleanchor="y",
    scaleratio=1,
    showticklabels=False,
    showgrid=False,
    zeroline=False,
    title=''
)
fig.update_yaxes(
    autorange="reversed",
    showticklabels=False,
    showgrid=False,
    zeroline=False,
    title=''
)
fig.update_layout(
    width=400,
    height=400,
    margin=dict(l=0, r=0, t=10, b=0),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.show()
# png_bytes = fig.to_image(format="png")
# display(Image(png_bytes))

In [ ]:

np_array = median_ro
valid_values = np_array[~np.isnan(np_array)]

perc = [0, 10, 25, 50, 75, 90, 100]
percentiles = np.percentile(valid_values, perc)

percentile_bins = np.digitize(np_array, percentiles, right=True) - 1
percentile_bins = np.clip(percentile_bins, 0, len(percentiles)-2)

percentile_bins = percentile_bins.astype(float)

flat_bins = percentile_bins.flatten()
flat_values = np_array.flatten()

mask = ~np.isnan(flat_bins) & ~np.isnan(flat_values)
flat_bins = flat_bins[mask].astype(int)
flat_values = flat_values[mask]

df = pd.DataFrame({
    'percentile_bin': flat_bins,
    'rainfall': flat_values
})

summary = df.groupby('percentile_bin')['rainfall'].describe()
summary.index = perc[1:]
summary

## Scoring

In [ ]:

perc = [0, 5, 10, 17.5, 25, 37.5, 50, 62.5, 75, 82.5, 90, 95, 100]

def compute_percentile_summary(np_array, var_name):
    valid_values = np_array[~np.isnan(np_array)]
    percentiles = np.percentile(valid_values, perc, method='weibull')

    bins = np.digitize(np_array, percentiles, right=True) - 1
    bins = np.clip(bins, 0, len(percentiles) - 2).astype(float)
    bins[np.isnan(np_array)] = np.nan

    flat_bins = bins.flatten()
    flat_values = np_array.flatten()

    mask = ~np.isnan(flat_bins) & ~np.isnan(flat_values)
    flat_bins = flat_bins[mask].astype(int)
    flat_values = flat_values[mask]

    df = pd.DataFrame({
        'percentile_bin': flat_bins,
        var_name: flat_values
    })

    summary = df.groupby('percentile_bin')[var_name].mean().to_frame()
    summary.index = [perc[i + 1] for i in summary.index]
    return summary

rainfall_summary = compute_percentile_summary(median_pr, 'rainfall')
runoff_summary = compute_percentile_summary(median_ro, 'runoff')

combined_summary = pd.concat([rainfall_summary, runoff_summary], axis=1)
combined_summary.index.name = 'Percentile'
combined_summary.T.round()

In [ ]:
# Categorical scoring
rainfall_bins = [98, 248, 353, 550, 753, 903, 1050, 1240, 1433, 1804, 2269, 3491]
runoff_bins = [6, 15, 21, 33, 74, 184, 293, 421, 553, 873, 1294, 2499]

def assign_scores(img, bins):
    num_classes = len(bins) + 1
    score = img.lte(bins[0]).multiply(1)
    for i in range(len(bins) - 1):
        lower = bins[i]
        upper = bins[i + 1]
        class_val = i + 2
        score = score.where(img.gt(lower).And(img.lte(upper)), class_val)
    score = score.where(img.gt(bins[-1]), num_classes)

    # Rescale from [1, num_classes] to [1, 5]
    score = score.subtract(1).multiply(4).divide(num_classes - 1).add(1)
    return score

rainfallScore = assign_scores(precipitationSum, rainfall_bins)
runoffScore = assign_scores(runoffSum, runoff_bins)

In [ ]:

hazard_scores = {
    'Rainfall': rainfallScore,
    'Runoff': runoffScore
}

hazard_scores_arrays = {}

for name, ee_image in hazard_scores.items():
    np_array = geemap.ee_to_numpy(
        ee_image.unmask(-9999),
        region=city_roi,
        scale=scale
    )
    np_array = np.squeeze(np_array)
    np_array = np_array.astype(float)
    np_array[np_array == -9999] = np.nan

    hazard_scores_arrays[name] = np_array

In [ ]:

custom_minmax = {
    'Rainfall': (1, 5),
    'Runoff': (1, 5)
}

keys = list(hazard_scores_arrays.keys())

frames = []
for key in keys:
    layer, scatter = create_layer_and_scatter(hazard_scores_arrays[key], gnylrd_cmap, key, custom_minmax)
    frames.append(go.Frame(
        data=[scatter] + boundary_traces,
        name=key,
        layout=dict(mapbox_layers=[layer])
    ))

initial_layer, initial_scatter = create_layer_and_scatter(hazard_scores_arrays[keys[0]], gnylrd_cmap, keys[0], custom_minmax)

buttons = []
for key in keys:
    buttons.append({
        'label': key,
        'method': 'animate',
        'args': [[key], {
            'frame': {'duration': 500, 'redraw': True},
            'mode': 'immediate',
            'transition': {'duration': 300}
        }]
    })

fig = go.Figure(
    data=[initial_scatter] + boundary_traces,
    frames=frames,
    layout=go.Layout(
        width=map_width,
        height=map_height,
        margin=dict(l=0, r=0, t=0, b=0),
        mapbox=dict(
            style='open-street-map',
            center=dict(lat=center_lat, lon=center_lon),
            zoom=zoom_level,
            layers=[initial_layer]
        ),
        updatemenus=[{
            'buttons': buttons,
            'direction': 'down',
            'showactive': True,
            'x': 0.02,
            'y': 0.91,
            'xanchor': 'left',
            'yanchor': 'bottom'
        }]
    )
)

fig.show()
# png_bytes = fig.to_image(format="png")
# display(Image(png_bytes))

In [ ]:
# Flood Hazard Score
floodHazardScore = rainfallScore.add(runoffScore).rename('Flood_Hazard')

# Rescale from [2, 10] to [1, 5]
floodHazardScore = floodHazardScore.subtract(2).multiply(4).divide(8).add(1)

# Flood hazard categories
tol = 0
floodHazardCategories = floodHazardScore.where(floodHazardScore.gt(4.2 - tol), 5)\
                                        .where(floodHazardScore.gt(3.4 - tol).And(floodHazardScore.lte(4.2 - tol)), 4)\
                                        .where(floodHazardScore.gt(2.6 - tol).And(floodHazardScore.lte(3.4 - tol)), 3)\
                                        .where(floodHazardScore.gt(1.8 - tol).And(floodHazardScore.lte(2.6 - tol)), 2)\
                                        .where(floodHazardScore.lte(1.8 - tol), 1)

In [ ]:

np_array = geemap.ee_to_numpy(
    floodHazardScore.unmask(-9999),
    region=city_roi,
    scale=scale,
)
np_array = np.squeeze(np_array)
np_array = np_array.astype(float)
np_array[np_array == -9999] = np.nan

min_val = 1
max_val = 5
array_scaled_for_cmap = (np_array - min_val) / (max_val - min_val)

colored = rainbow_cmap(array_scaled_for_cmap)
alpha_channel = np.where(np.isnan(np_array), 0, 255).astype(np.uint8)
rgb_array = (colored[:, :, :3] * 255).astype(np.uint8)
rgba_array = np.dstack((rgb_array, alpha_channel))
image = PILImage.fromarray(rgba_array)
buffer = io.BytesIO()
image.save(buffer, format='PNG')
buffer.seek(0)
encoded_image = "data:image/png;base64," + base64.b64encode(buffer.read()).decode()

lon_min, lon_max, lat_min, lat_max = extent 

image_coordinates = [
    [lon_min, lat_max],
    [lon_max, lat_max],
    [lon_max, lat_min],
    [lon_min, lat_min],
]

layer = dict(
    sourcetype='image',
    source=encoded_image,
    coordinates=image_coordinates,
    opacity=1
)

nrows, ncols = np_array.shape
lons = np.linspace(extent[0], extent[1], ncols)
lats = np.linspace(extent[3], extent[2], nrows)

lon_grid, lat_grid = np.meshgrid(lons, lats)

flat_lons = lon_grid.flatten()
flat_lats = lat_grid.flatten()
flat_vals = np_array.flatten()

valid_mask = ~np.isnan(flat_vals)
flat_lons = flat_lons[valid_mask]
flat_lats = flat_lats[valid_mask]
flat_vals = flat_vals[valid_mask]

hover_texts = [f"Value: {v:.2f}" for v in flat_vals]

fig = go.Figure()

fig.add_trace(go.Scattermapbox(
    lat=flat_lats,
    lon=flat_lons,
    mode='markers',
    marker=dict(size=5, color='rgba(0,0,0,0)'),
    hoverinfo='text',
    hovertext=hover_texts,
    showlegend=False,
))

fig.update_layout(
    width=map_width,
    height=map_height,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style='open-street-map',
        center=dict(lat=center_lat, lon=center_lon),
        zoom=zoom_level * 1.0,
        layers=[layer]
    )
)

fig.add_traces(boundary_traces)

fig.show()
# png_bytes = fig.to_image(format="png")
# display(Image(png_bytes))

In [ ]:

np_array = geemap.ee_to_numpy(
    floodHazardCategories.unmask(-9999),
    region=city_roi,
    scale=scale,
)
np_array = np.squeeze(np_array)
np_array = np_array.astype(float)
np_array[np_array == -9999] = np.nan

min_val = 1
max_val = 5
array_scaled_for_cmap = (np_array - min_val) / (max_val - min_val)

colored = rainbow_cmap(array_scaled_for_cmap)
alpha_channel = np.where(np.isnan(np_array), 0, 255).astype(np.uint8)
rgb_array = (colored[:, :, :3] * 255).astype(np.uint8)
rgba_array = np.dstack((rgb_array, alpha_channel))
image = PILImage.fromarray(rgba_array)
buffer = io.BytesIO()
image.save(buffer, format='PNG')
buffer.seek(0)
encoded_image = "data:image/png;base64," + base64.b64encode(buffer.read()).decode()

lon_min, lon_max, lat_min, lat_max = extent 

image_coordinates = [
    [lon_min, lat_max],
    [lon_max, lat_max],
    [lon_max, lat_min],
    [lon_min, lat_min],
]

layer = dict(
    sourcetype='image',
    source=encoded_image,
    coordinates=image_coordinates,
    opacity=1
)

nrows, ncols = np_array.shape
lons = np.linspace(extent[0], extent[1], ncols)
lats = np.linspace(extent[3], extent[2], nrows)

lon_grid, lat_grid = np.meshgrid(lons, lats)

flat_lons = lon_grid.flatten()
flat_lats = lat_grid.flatten()
flat_vals = np_array.flatten()

valid_mask = ~np.isnan(flat_vals)
flat_lons = flat_lons[valid_mask]
flat_lats = flat_lats[valid_mask]
flat_vals = flat_vals[valid_mask]

hover_texts = [f"Value: {v:.2f}" for v in flat_vals]

fig = go.Figure()

fig.add_trace(go.Scattermapbox(
    lat=flat_lats,
    lon=flat_lons,
    mode='markers',
    marker=dict(size=5, color='rgba(0,0,0,0)'),
    hoverinfo='text',
    hovertext=hover_texts,
    showlegend=False,
))

fig.update_layout(
    width=map_width,
    height=map_height,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style='open-street-map',
        center=dict(lat=center_lat, lon=center_lon),
        zoom=zoom_level * 1.0,
        layers=[layer]
    )
)

fig.add_traces(boundary_traces)

fig.show()
# png_bytes = fig.to_image(format="png")
# display(Image(png_bytes))

# Vulnerability

## Definition

In [ ]:
# Import datasets
gsw = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')
srtm = ee.Image("USGS/SRTMGL1_003")
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

In [ ]:
# Permanent water
permanent = gsw.select('occurrence').gt(80).selfMask()

# Distance from permanent water
pxLen = ee.Image.pixelArea().sqrt()
distance = permanent.fastDistanceTransform(1024).sqrt().multiply(pxLen).clip(city_roi)
distance = distance.reproject('EPSG:4326', scale=30)
distance = distance.updateMask(distance.neq(0).And(srtm.mask())) # masking water bodies

# Elevation
elevation = srtm.clip(city_roi)

# Topographic Position Index
tpi = elevation.subtract(
    elevation.focalMean(5).reproject(crs='EPSG:4326', scale=30)
).rename('TPI')

def cloudMask(image):
    qa = image.select('QA_PIXEL')
    dilated = 1 << 1
    cirrus = 1 << 2
    cloud = 1 << 3
    shadow = 1 << 4
    mask = (
        qa.bitwiseAnd(dilated).eq(0)
        .And(qa.bitwiseAnd(cirrus).eq(0))
        .And(qa.bitwiseAnd(cloud).eq(0))
        .And(qa.bitwiseAnd(shadow).eq(0))
    )
    return image.select(['SR_B.*'], ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'])\
                .multiply(0.0000275).add(-0.2).updateMask(mask)

landsat8 = l8.filterBounds(city_roi)\
           .filterDate(f'{landsat_year}-01-01', f'{landsat_year}-12-31')\
           .map(cloudMask)\
           .median()\
           .clip(city_roi)

bandMap = {
    'RED': landsat8.select('B4'),
    'NIR': landsat8.select('B5'),
    'GREEN': landsat8.select('B3')
}

# NDVI
ndvi = landsat8.expression('(NIR - RED) / (NIR + RED)', bandMap).rename('NDVI')

# NDWI
ndwi = landsat8.expression('(GREEN - NIR) / (GREEN + NIR)', bandMap).rename('NDWI')

In [ ]:

vulnerability_data = {
    'Distance': distance,
    'DEM': elevation,
    'TPI': tpi,
    'NDVI': ndvi,
    'NDWI': ndwi,
}

vulnerability_arrays = {}

for name, ee_image in vulnerability_data.items():
    arr = geemap.ee_to_numpy(
        ee_image.unmask(-9999),
        region=city_roi,
        scale=scale
    )
    arr = np.squeeze(arr)
    arr = arr.astype(float)
    arr[arr == -9999] = np.nan

    vulnerability_arrays[name] = arr

In [ ]:

pd.DataFrame({
    name: pd.Series(arr.ravel()[~np.isnan(arr.ravel())]).describe()
    for name, arr in vulnerability_arrays.items()
})

In [ ]:

custom_minmax = {
    'NDVI': (-1, 1),
    'NDWI': (-1, 1),
    'Distance': (0, 5000)
}

keys = list(vulnerability_arrays.keys())

frames = []
for key in keys:
    layer, scatter = create_layer_and_scatter(vulnerability_arrays[key], rainbow_cmap, key=key, custom_minmax=custom_minmax)
    frames.append(go.Frame(
        data=[scatter] + boundary_traces,
        name=key,
        layout=dict(mapbox_layers=[layer])
    ))

initial_layer, initial_scatter = create_layer_and_scatter(vulnerability_arrays[keys[0]], rainbow_cmap, key=keys[0], custom_minmax=custom_minmax)

buttons = []
for key in keys:
    buttons.append({
        'label': key,
        'method': 'animate',
        'args': [[key], {
            'frame': {'duration': 500, 'redraw': True},
            'mode': 'immediate',
            'transition': {'duration': 300}
        }]
    })

fig = go.Figure(
    data=[initial_scatter] + boundary_traces,
    frames=frames,
    layout=go.Layout(
        width=map_width,
        height=map_height,
        margin=dict(l=0, r=0, t=0, b=0),
        mapbox=dict(
            style='open-street-map',
            center=dict(lat=center_lat, lon=center_lon),
            zoom=zoom_level,
            layers=[initial_layer]
        ),
        updatemenus=[{
            'buttons': buttons,
            'direction': 'down',
            'showactive': True,
            'x': 0.02, 'y': 0.91,
            'xanchor': 'left',
            'yanchor': 'bottom'
        }]
    )
)

fig.show()
# png_bytes = fig.to_image(format="png")
# display(Image(png_bytes))

In [ ]:
# # Plotting using geemap
# m = geemap.Map()
# m.centerObject(city_roi, zoom=None)

# m.addLayer(
#     city_roi.style(color='black', fillColor='00000000', width=1),
#     {},
#     'City ROI'
# )

# vulnerability_data = {
#     'Distance': distance,
#     'DEM': elevation,
#     'TPI': tpi,
#     'NDVI': ndvi,
#     'NDWI': ndwi
# }

# vis_params = {
#     'Distance': {
#         'min': 0,
#         'max': 5000,
#         'palette': ['blue', 'cyan', 'green', 'yellow', 'red']
#     },
#     'DEM': {
#         'min': 0,
#         'max': 100,
#         'palette': ['green', 'yellow', 'red', 'white']
#     },
#     'TPI': {
#         'min': -5,
#         'max': 5,
#         'palette': ['blue', 'yellow', 'red']
#     },
#     'NDVI': {
#         'min': -1,
#         'max': 1,
#         'palette': ['blue', 'white', 'green']
#     },
#     'NDWI': {
#         'min': -1,
#         'max': 1,
#         'palette': ['red', 'white', 'blue']
#     }
# }

# for name, image in list(vulnerability_data.items()):
#     m.addLayer(image, vis_params[name], name)
# m

## Scoring

In [ ]:
# Categorical scoring
distanceScore = distance.where(distance.gt(4000), 1)\
                .where(distance.gt(3000).And(distance.lte(4000)), 2)\
                .where(distance.gt(2000).And(distance.lte(3000)), 3)\
                .where(distance.gt(1000).And(distance.lte(2000)), 4)\
                .where(distance.lte(1000), 5)

elevScore = elevation.updateMask(distance.neq(0))\
            .where(elevation.gt(20), 1)\
            .where(elevation.gt(15).And(elevation.lte(20)), 2)\
            .where(elevation.gt(10).And(elevation.lte(15)), 3)\
            .where(elevation.gt(5).And(elevation.lte(10)), 4)\
            .where(elevation.lte(5), 5)

topoScore = tpi.updateMask(distance.neq(0))\
            .where(tpi.gt(0), 1)\
            .where(tpi.gt(-2).And(tpi.lte(0)), 2)\
            .where(tpi.gt(-4).And(tpi.lte(-2)), 3)\
            .where(tpi.gt(-6).And(tpi.lte(-4)), 4)\
            .where(tpi.lte(-6), 5)

vegScore = ndvi.updateMask(distance.neq(0))\
           .where(ndvi.gt(0.8), 1)\
           .where(ndvi.gt(0.6).And(ndvi.lte(0.8)), 2)\
           .where(ndvi.gt(0.4).And(ndvi.lte(0.6)), 3)\
           .where(ndvi.gt(0.2).And(ndvi.lte(0.4)), 4)\
           .where(ndvi.lte(0.2), 5)

wetScore = ndwi.updateMask(distance.neq(0))\
           .where(ndwi.gt(0.6), 5)\
           .where(ndwi.gt(0.2).And(ndwi.lte(0.6)), 4)\
           .where(ndwi.gt(-0.2).And(ndwi.lte(0.2)), 3)\
           .where(ndwi.gt(-0.6).And(ndwi.lte(-0.2)), 2)\
           .where(ndwi.lte(-0.6), 1)

In [ ]:

vulnerability_scores = {
    'Distance': distanceScore,
    'DEM': elevScore,
    'TPI' : topoScore,
    'NDVI': vegScore,
    'NDWI': wetScore
}

vulnerability_scores_arrays = {}

for name, ee_image in vulnerability_scores.items():
    arr = geemap.ee_to_numpy(
        ee_image.unmask(-9999),
        region=city_roi,
        scale=scale
    )
    arr = np.squeeze(arr)
    arr = arr.astype(float)
    arr[arr == -9999] = np.nan

    vulnerability_scores_arrays[name] = arr

In [ ]:

custom_minmax = {
    'Distance': (1, 5),
    'DEM': (1, 5),
    'TPI': (1, 5),
    'NDVI': (1, 5),
    'NDWI': (1, 5)
}

keys = list(vulnerability_scores_arrays.keys())

frames = []
for key in keys:
    layer, scatter = create_layer_and_scatter(vulnerability_scores_arrays[key], rainbow_cmap, key, custom_minmax)
    frames.append(go.Frame(
        data=[scatter] + boundary_traces,
        name=key,
        layout=dict(mapbox_layers=[layer])
    ))

initial_layer, initial_scatter = create_layer_and_scatter(vulnerability_scores_arrays[keys[0]], rainbow_cmap, keys[0], custom_minmax)

buttons = []
for key in keys:
    buttons.append({
        'label': key,
        'method': 'animate',
        'args': [[key], {
            'frame': {'duration': 500, 'redraw': True},
            'mode': 'immediate',
            'transition': {'duration': 300}
        }]
    })

fig = go.Figure(
    data=[initial_scatter] + boundary_traces,
    frames=frames,
    layout=go.Layout(
        width=map_width,
        height=map_height,
        margin=dict(l=0, r=0, t=0, b=0),
        mapbox=dict(
            style='open-street-map',
            center=dict(lat=center_lat, lon=center_lon),
            zoom=zoom_level,
            layers=[initial_layer]
        ),
        updatemenus=[{
            'buttons': buttons,
            'direction': 'down',
            'showactive': True,
            'x': 0.02,
            'y': 0.91,
            'xanchor': 'left',
            'yanchor': 'bottom'
        }]
    )
)

fig.show()
# png_bytes = fig.to_image(format="png")
# display(Image(png_bytes))

In [ ]:
# Flood vulnerability score
floodVulnerabilityScore = (distanceScore.multiply(0.25)).add(topoScore).add(vegScore).add(wetScore).add(elevScore).rename('Flood_Vulnerability')

# scale to from 4.5-22.5 to 1-5
floodVulnerabilityScore = floodVulnerabilityScore \
                          .subtract(4.5) \
                          .multiply(4 / 18) \
                          .add(1)

# Flood vulnerability categories
tol = 0
floodVulnerabilityCategories = floodVulnerabilityScore.where(floodVulnerabilityScore.gt(4.2 - tol), 5)\
                                                      .where(floodVulnerabilityScore.gt(3.4 - tol).And(floodVulnerabilityScore.lte(4.2 - tol)), 4)\
                                                      .where(floodVulnerabilityScore.gt(2.6 - tol).And(floodVulnerabilityScore.lte(3.4 - tol)), 3)\
                                                      .where(floodVulnerabilityScore.gt(1.8 - tol).And(floodVulnerabilityScore.lte(2.6 - tol)), 2)\
                                                      .where(floodVulnerabilityScore.lte(1.8 - tol), 1)

In [ ]:

np_array = geemap.ee_to_numpy(
    floodVulnerabilityScore.unmask(-9999),
    region=city_roi,
    scale=scale,
)
np_array = np.squeeze(np_array)
np_array = np_array.astype(float)
np_array[np_array == -9999] = np.nan

min_val = 1
max_val = 5
array_scaled_for_cmap = (np_array - min_val) / (max_val - min_val)

colored = rainbow_cmap(array_scaled_for_cmap)
alpha_channel = np.where(np.isnan(np_array), 0, 255).astype(np.uint8)
rgb_array = (colored[:, :, :3] * 255).astype(np.uint8)
rgba_array = np.dstack((rgb_array, alpha_channel))
image = PILImage.fromarray(rgba_array)
buffer = io.BytesIO()
image.save(buffer, format='PNG')
buffer.seek(0)
encoded_image = "data:image/png;base64," + base64.b64encode(buffer.read()).decode()

lon_min, lon_max, lat_min, lat_max = extent

image_coordinates = [
    [lon_min, lat_max],
    [lon_max, lat_max],
    [lon_max, lat_min],
    [lon_min, lat_min],
]

layer = dict(
    sourcetype='image',
    source=encoded_image,
    coordinates=image_coordinates,
    opacity=1
)

center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2

zoom_level = get_zoom_level([lon_min, lat_min, lon_max, lat_max], map_width, map_height)

nrows, ncols = np_array.shape
lons = np.linspace(lon_min, lon_max, ncols)
lats = np.linspace(lat_max, lat_min, nrows)

lon_grid, lat_grid = np.meshgrid(lons, lats)

flat_lons = lon_grid.flatten()
flat_lats = lat_grid.flatten()
flat_vals = np_array.flatten()

valid_mask = ~np.isnan(flat_vals)
flat_lons = flat_lons[valid_mask]
flat_lats = flat_lats[valid_mask]
flat_vals = flat_vals[valid_mask]

hover_texts = [f"Value: {v:.2f}" for v in flat_vals]

fig = go.Figure()

fig.add_trace(go.Scattermapbox(
    lat=flat_lats,
    lon=flat_lons,
    mode='markers',
    marker=dict(size=5, color='rgba(0,0,0,0)'),
    hoverinfo='text',
    hovertext=hover_texts,
    showlegend=False,
))

fig.update_layout(
    width=map_width,
    height=map_height,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style='open-street-map',
        center=dict(lat=center_lat, lon=center_lon),
        zoom=zoom_level * 1.0,
        layers=[layer]
    )
)

fig.add_traces(boundary_traces)

fig.show()
# png_bytes = fig.to_image(format="png")
# display(Image(png_bytes))

In [ ]:

np_array = geemap.ee_to_numpy(
    floodVulnerabilityCategories.unmask(-9999),
    region=city_roi,
    scale=scale,
)
np_array = np.squeeze(np_array)
np_array = np_array.astype(float)
np_array[np_array == -9999] = np.nan

min_val = 1
max_val = 5
array_scaled_for_cmap = (np_array - min_val) / (max_val - min_val)

colored = rainbow_cmap(array_scaled_for_cmap)
alpha_channel = np.where(np.isnan(np_array), 0, 255).astype(np.uint8)
rgb_array = (colored[:, :, :3] * 255).astype(np.uint8)
rgba_array = np.dstack((rgb_array, alpha_channel))
image = PILImage.fromarray(rgba_array)
buffer = io.BytesIO()
image.save(buffer, format='PNG')
buffer.seek(0)
encoded_image = "data:image/png;base64," + base64.b64encode(buffer.read()).decode()

lon_min, lon_max, lat_min, lat_max = extent

image_coordinates = [
    [lon_min, lat_max],
    [lon_max, lat_max],
    [lon_max, lat_min],
    [lon_min, lat_min],
]

layer = dict(
    sourcetype='image',
    source=encoded_image,
    coordinates=image_coordinates,
    opacity=1
)

center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2

zoom_level = get_zoom_level([lon_min, lat_min, lon_max, lat_max], map_width, map_height)

nrows, ncols = np_array.shape
lons = np.linspace(lon_min, lon_max, ncols)
lats = np.linspace(lat_max, lat_min, nrows)

lon_grid, lat_grid = np.meshgrid(lons, lats)

flat_lons = lon_grid.flatten()
flat_lats = lat_grid.flatten()
flat_vals = np_array.flatten()

valid_mask = ~np.isnan(flat_vals)
flat_lons = flat_lons[valid_mask]
flat_lats = flat_lats[valid_mask]
flat_vals = flat_vals[valid_mask]

hover_texts = [f"Value: {v:.2f}" for v in flat_vals]

fig = go.Figure()

fig.add_trace(go.Scattermapbox(
    lat=flat_lats,
    lon=flat_lons,
    mode='markers',
    marker=dict(size=5, color='rgba(0,0,0,0)'),
    hoverinfo='text',
    hovertext=hover_texts,
    showlegend=False,
))

fig.update_layout(
    width=map_width,
    height=map_height,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style='open-street-map',
        center=dict(lat=center_lat, lon=center_lon),
        zoom=zoom_level * 1.0,
        layers=[layer]
    )
)

fig.add_traces(boundary_traces)

fig.show()
# png_bytes = fig.to_image(format="png")
# display(Image(png_bytes))

# Exposure

## Definition

In [ ]:
# Import dataset
pop = ee.ImageCollection("JRC/GHSL/P2023A/GHS_POP")

In [ ]:
# Population
pop_img = pop.filterDate(f'{pop_year}-01-01', f'{pop_year}-12-31').first()
pop_img = pop_img.clip(city_roi).reproject(crs='EPSG:4326', scale=30)

In [ ]:

exposure_data = {
    'Population': pop_img
}

exposure_arrays = {}

for name, ee_image in exposure_data.items():
    arr = geemap.ee_to_numpy(
        ee_image.unmask(-9999),
        region=city_roi,
        scale=scale
    )
    arr = np.squeeze(arr)
    arr = arr.astype(float)
    arr[arr == -9999] = np.nan

    exposure_arrays[name] = arr

In [ ]:

pd.DataFrame({
    name: pd.Series(arr.ravel()[~np.isnan(arr.ravel())]).describe()
    for name, arr in exposure_arrays.items()
})

In [ ]:

exposure_data = {
    'Population': pop_img
}

exposure_arrays = {}

for name, ee_image in exposure_data.items():
    np_array = geemap.ee_to_numpy(
        ee_image.unmask(-9999),
        region=city_roi,
        scale=scale
    )
    np_array = np.squeeze(np_array)
    np_array = np_array.astype(float)
    np_array[np_array == -9999] = np.nan

    exposure_arrays[name] = np_array

In [ ]:

custom_minmax = {
    'Population': (0, 1000)
}

keys = list(exposure_arrays.keys())

frames = []
for key in keys:
    layer, scatter = create_layer_and_scatter(exposure_arrays[key], gnylrd_cmap, key, custom_minmax)
    frames.append(go.Frame(
        data=[scatter] + boundary_traces,
        name=key,
        layout=dict(mapbox_layers=[layer])
    ))

initial_layer, initial_scatter = create_layer_and_scatter(exposure_arrays[keys[0]], gnylrd_cmap, keys[0], custom_minmax)

buttons = []
for key in keys:
    buttons.append({
        'label': key,
        'method': 'animate',
        'args': [[key], {
            'frame': {'duration': 500, 'redraw': True},
            'mode': 'immediate',
            'transition': {'duration': 300}
        }]
    })

fig = go.Figure(
    data=[initial_scatter] + boundary_traces,
    frames=frames,
    layout=go.Layout(
        width=map_width,
        height=map_height,
        margin=dict(l=0, r=0, t=0, b=0),
        mapbox=dict(
            style='open-street-map',
            center=dict(lat=center_lat, lon=center_lon),
            zoom=zoom_level,
            layers=[initial_layer]
        ),
        updatemenus=[{
            'buttons': buttons,
            'direction': 'down',
            'showactive': True,
            'x': 0.02,
            'y': 0.91,
            'xanchor': 'left',
            'yanchor': 'bottom'
        }]
    )
)

fig.show()
# png_bytes = fig.to_image(format="png")
# display(Image(png_bytes))

## Analysis

In [ ]:
pop = ee.ImageCollection("JRC/GHSL/P2023A/GHS_POP")

In [ ]:
# Top 10 populated smart cities
cities = ['Delhi', 'Kolkata', 'Chennai', 'Bangalore', 'Ahmedabad', 'Pune', 'Kanpur', 'Surat', 'Lucknow', 'Nagpur']

In [ ]:
# Total population of top 10 populated smart cities
median_pop_dict = {}

for city_name in cities:
    cityROI = ee.FeatureCollection(f'projects/ee-smartcities56/assets/{city_name}')
    
    years = list(range(1975, 2030, 5))  # 1975 to 2025 inclusive
    
    year_pops = []
    
    for y in years:
        img = pop.filterDate(
            ee.Date.fromYMD(y, 1, 1),
            ee.Date.fromYMD(y, 12, 31)
        ).first()
        
        # Total population for the year
        pop_sum = img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=cityROI.geometry(),
            scale=100,
            maxPixels=1e12
        ).get('population_count')
        
        year_pops.append(ee.Number(pop_sum))
    
    # Median population over years
    median_pop = ee.Number(ee.List(year_pops).reduce(ee.Reducer.median())).getInfo()
    
    median_pop_dict[city_name] = median_pop
    
pd.Series(median_pop_dict).astype(int).to_frame('Population').sort_values('Population', ascending=False)

In [ ]:
# Top 10 populated Indian cities - generalises better
cities = ['Delhi', 'Kolkata', 'Chennai', 'Bangalore', 'Ahmedabad', 'Pune', 'Kanpur', 'Surat', 'Lucknow', 'Nagpur']

In [ ]:
# Total population of top 10 populated Indian cities
population_matrix_arrays = []

for city in cities:
  arrays = []
  city_roi_feature = ee.FeatureCollection(f"projects/ee-539srijansiddharth/assets/Smart-Cities-India/{city}")

  for year in range(1975, 2020, 5):
    pop_city_data = pop.filterDate(f"{year}-01-01", f"{year}-12-31") \
                 .first() \
                 .clip(city_roi_feature)

    arr = geemap.ee_to_numpy(
        pop_city_data.unmask(-9999),
        region=city_roi_feature,
        scale=30,
    )

    arr = np.squeeze(arr).astype(float)
    arr[(arr == -9999)] = np.nan
    arrays.append(arr)
  population_matrix_arrays.append(np.stack(arrays, axis=0))

In [ ]:

percentiles_list = [0, 10, 25, 50, 75, 90, 100]

percentile_values = {
    city: np.percentile(
        np.percentile(population_matrix_arrays[i], 50, axis=0)
        [~np.isnan(np.percentile(population_matrix_arrays[i], 50, axis=0))],
        percentiles_list
    )
    for i, city in enumerate(cities)
}

pd.DataFrame(percentile_values, index=percentiles_list)

In [ ]:
# Median population over years
percentiles_list = [0, 10, 25, 50, 75, 90, 100]

percentile_values = {
    i: np.percentile(np.percentile(population_matrix_arrays[i], 50, axis=0)[~np.isnan(np.percentile(population_matrix_arrays[i], 50, axis=0))], percentiles_list)
    for i in range(len(cities))
}
pd.DataFrame(percentile_values, index=percentiles_list).agg(['median'], axis=1)

## Scoring

The population is very highly skewed, thus considering only 10 most densely populated cities from India and their percentiles at every ~10% from 55th percentile, a scoring can be prepared for population.

In [ ]:

city_arrays = [np.percentile(population_matrix_arrays[i], 50, axis=0) for i in range(len(cities))]
percentiles_list = [0, 55, 65, 75, 85, 95, 97.5, 99, 99.5, 99.75, 99.9, 100]

percentile_values = {
    i: np.percentile(city_arrays[i].ravel()[~np.isnan(city_arrays[i].ravel())], percentiles_list)
    for i in range(len(cities))
}
pd.DataFrame(percentile_values, index=percentiles_list).agg(['median'], axis=1).T.round().astype(int)

In [ ]:
# Categorical scoring
bins = [57, 117, 183, 241, 408, 492, 607, 697, 793, 965, 1527]

popScore = pop_img.lt(bins[0]).multiply(1)

for i in range(len(bins) - 1):
    lower = bins[i]
    upper = bins[i + 1]
    class_value = i + 2
    popScore = popScore.where(pop_img.gte(lower).And(pop_img.lt(upper)), class_value)

popScore = popScore.where(pop_img.gte(bins[-1]), len(bins) + 1) # last bin

# rescale to 1-5
popScore = popScore.subtract(1) \
                   .multiply(5 - 1) \
                   .divide(len(bins) + 1 - 1) \
                   .add(1) \
                   .min(4.25)

In [ ]:

exposure_scores = {
    'Population': popScore
}

exposure_scores_arrays = {}

for name, ee_image in exposure_scores.items():
    np_array = geemap.ee_to_numpy(
        ee_image.unmask(-9999),
        region=city_roi,
        scale=scale
    )
    np_array = np.squeeze(np_array)
    np_array = np_array.astype(float)
    np_array[np_array == -9999] = np.nan

    exposure_scores_arrays[name] = np_array

In [ ]:

custom_minmax = {
    'Population': (1, 5)
}

keys = list(exposure_scores_arrays.keys())

frames = []
for key in keys:
    layer, scatter = create_layer_and_scatter(exposure_scores_arrays[key], gnylrd_cmap, key, custom_minmax)
    frames.append(go.Frame(
        data=[scatter] + boundary_traces,
        name=key,
        layout=dict(mapbox_layers=[layer])
    ))

initial_layer, initial_scatter = create_layer_and_scatter(exposure_scores_arrays[keys[0]], gnylrd_cmap, keys[0], custom_minmax)

buttons = []
for key in keys:
    buttons.append({
        'label': key,
        'method': 'animate',
        'args': [[key], {
            'frame': {'duration': 500, 'redraw': True},
            'mode': 'immediate',
            'transition': {'duration': 300}
        }]
    })

fig = go.Figure(
    data=[initial_scatter] + boundary_traces,
    frames=frames,
    layout=go.Layout(
        width=map_width,
        height=map_height,
        margin=dict(l=0, r=0, t=0, b=0),
        mapbox=dict(
            style='open-street-map',
            center=dict(lat=center_lat, lon=center_lon),
            zoom=zoom_level,
            layers=[initial_layer]
        ),
        updatemenus=[{
            'buttons': buttons,
            'direction': 'down',
            'showactive': True,
            'x': 0.02,
            'y': 0.91,
            'xanchor': 'left',
            'yanchor': 'bottom'
        }]
    )
)

fig.show()
# png_bytes = fig.to_image(format="png")
# display(Image(png_bytes))

In [ ]:
# Flood exposure score
floodExposureScore = popScore.rename('Flood_Exposure')

# Flood exposure categories
tol = 0
floodExposureCategories = floodExposureScore.where(floodExposureScore.gt(4.2 - tol), 5)\
                                            .where(floodExposureScore.gt(3.4 - tol).And(floodExposureScore.lte(4.2 - tol)), 4)\
                                            .where(floodExposureScore.gt(2.6 - tol).And(floodExposureScore.lte(3.4 - tol)), 3)\
                                            .where(floodExposureScore.gt(1.8 - tol).And(floodExposureScore.lte(2.6 - tol)), 2)\
                                            .where(floodExposureScore.lte(1.8 - tol), 1)

In [ ]:

np_array = geemap.ee_to_numpy(
    floodExposureScore.unmask(-9999),
    region=city_roi,
    scale=scale,
)
np_array = np.squeeze(np_array)
np_array = np_array.astype(float)
np_array[np_array == -9999] = np.nan

min_val = 1
max_val = 5
array_scaled_for_cmap = (np_array - min_val) / (max_val - min_val)

colored = LinearSegmentedColormap.from_list("custom_scale", ['violet', 'blue', 'green', 'gold', 'red'])(array_scaled_for_cmap)
alpha_channel = np.where(np.isnan(np_array), 0, 255).astype(np.uint8)
rgb_array = (colored[:, :, :3] * 255).astype(np.uint8)
rgba_array = np.dstack((rgb_array, alpha_channel))
image = PILImage.fromarray(rgba_array)
buffer = io.BytesIO()
image.save(buffer, format='PNG')
buffer.seek(0)
encoded_image = "data:image/png;base64," + base64.b64encode(buffer.read()).decode()

lon_min, lon_max, lat_min, lat_max = extent

image_coordinates = [
    [lon_min, lat_max],
    [lon_max, lat_max],
    [lon_max, lat_min],
    [lon_min, lat_min],
]

layer = dict(
    sourcetype='image',
    source=encoded_image,
    coordinates=image_coordinates,
    opacity=1
)

nrows, ncols = np_array.shape
lons = np.linspace(extent[0], extent[1], ncols)
lats = np.linspace(extent[3], extent[2], nrows)

lon_grid, lat_grid = np.meshgrid(lons, lats)

flat_lons = lon_grid.flatten()
flat_lats = lat_grid.flatten()
flat_vals = np_array.flatten()

valid_mask = ~np.isnan(flat_vals)
flat_lons = flat_lons[valid_mask]
flat_lats = flat_lats[valid_mask]
flat_vals = flat_vals[valid_mask]

hover_texts = [f"Value: {v:.2f}" for v in flat_vals]

fig = go.Figure()

fig.add_trace(go.Scattermapbox(
    lat=flat_lats,
    lon=flat_lons,
    mode='markers',
    marker=dict(size=5, color='rgba(0,0,0,0)'),
    hoverinfo='text',
    hovertext=hover_texts,
    showlegend=False,
))

fig.update_layout(
    width=map_width,
    height=map_height,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style='open-street-map',
        center=dict(lat=center_lat, lon=center_lon),
        zoom=zoom_level * 1.0,
        layers=[layer]
    )
)

fig.add_traces(boundary_traces)

fig.show()
# png_bytes = fig.to_image(format="png")
# display(Image(png_bytes))

In [ ]:

np_array = geemap.ee_to_numpy(
    floodExposureCategories.unmask(-9999),
    region=city_roi,
    scale=scale,
)
np_array = np.squeeze(np_array)
np_array = np_array.astype(float)
np_array[np_array == -9999] = np.nan

min_val = 1
max_val = 5
array_scaled_for_cmap = (np_array - min_val) / (max_val - min_val)

colored = LinearSegmentedColormap.from_list("custom_scale", ['violet', 'blue', 'green', 'gold', 'red'])(array_scaled_for_cmap)
alpha_channel = np.where(np.isnan(np_array), 0, 255).astype(np.uint8)
rgb_array = (colored[:, :, :3] * 255).astype(np.uint8)
rgba_array = np.dstack((rgb_array, alpha_channel))
image = PILImage.fromarray(rgba_array)
buffer = io.BytesIO()
image.save(buffer, format='PNG')
buffer.seek(0)
encoded_image = "data:image/png;base64," + base64.b64encode(buffer.read()).decode()

lon_min, lon_max, lat_min, lat_max = extent

image_coordinates = [
    [lon_min, lat_max],
    [lon_max, lat_max],
    [lon_max, lat_min],
    [lon_min, lat_min],
]

layer = dict(
    sourcetype='image',
    source=encoded_image,
    coordinates=image_coordinates,
    opacity=1
)

nrows, ncols = np_array.shape
lons = np.linspace(extent[0], extent[1], ncols)
lats = np.linspace(extent[3], extent[2], nrows)

lon_grid, lat_grid = np.meshgrid(lons, lats)

flat_lons = lon_grid.flatten()
flat_lats = lat_grid.flatten()
flat_vals = np_array.flatten()

valid_mask = ~np.isnan(flat_vals)
flat_lons = flat_lons[valid_mask]
flat_lats = flat_lats[valid_mask]
flat_vals = flat_vals[valid_mask]

hover_texts = [f"Value: {v:.2f}" for v in flat_vals]

fig = go.Figure()

fig.add_trace(go.Scattermapbox(
    lat=flat_lats,
    lon=flat_lons,
    mode='markers',
    marker=dict(size=5, color='rgba(0,0,0,0)'),
    hoverinfo='text',
    hovertext=hover_texts,
    showlegend=False,
))

fig.update_layout(
    width=map_width,
    height=map_height,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style='open-street-map',
        center=dict(lat=center_lat, lon=center_lon),
        zoom=zoom_level * 1.0,
        layers=[layer]
    )
)

fig.add_traces(boundary_traces)

fig.show()
# png_bytes = fig.to_image(format="png")
# display(Image(png_bytes))

# Risk

In [ ]:
# Flood risk score
floodRiskScore = floodVulnerabilityScore.multiply(floodHazardScore).multiply(floodExposureScore).rename('Risk')

# Flood risk categories
riskMax = 50
tol = 2.5
floodRiskCategories = floodRiskScore.where(floodRiskScore.gt(riskMax - tol), 5)\
                                    .where(floodRiskScore.gt(3 * riskMax / 4 - tol).And(floodRiskScore.lte(riskMax - tol)), 4)\
                                    .where(floodRiskScore.gt(2 * riskMax / 4 - tol).And(floodRiskScore.lte(3 * riskMax / 4 - tol)), 3)\
                                    .where(floodRiskScore.gt(riskMax / 4 - tol).And(floodRiskScore.lte(2 * riskMax / 4 - tol)), 2)\
                                    .where(floodRiskScore.lte(riskMax / 4 - tol), 1)

In [ ]:

np_array = geemap.ee_to_numpy(
    floodRiskScore.unmask(-9999),
    region=city_roi,
    scale=scale,
)
np_array = np.squeeze(np_array)
np_array = np_array.astype(float)
np_array[np_array == -9999] = np.nan

min_val = 1
max_val = 45
array_scaled_for_cmap = (np_array - min_val) / (max_val - min_val)

colored = rainbow_cmap(array_scaled_for_cmap)
alpha_channel = np.where(np.isnan(np_array), 0, 255).astype(np.uint8)
rgb_array = (colored[:, :, :3] * 255).astype(np.uint8)
rgba_array = np.dstack((rgb_array, alpha_channel))
image = PILImage.fromarray(rgba_array)
buffer = io.BytesIO()
image.save(buffer, format='PNG')
buffer.seek(0)
encoded_image = "data:image/png;base64," + base64.b64encode(buffer.read()).decode()

lon_min, lon_max, lat_min, lat_max = extent

image_coordinates = [
    [lon_min, lat_max],
    [lon_max, lat_max],
    [lon_max, lat_min],
    [lon_min, lat_min],
]

layer = dict(
    sourcetype='image',
    source=encoded_image,
    coordinates=image_coordinates,
    opacity=1
)

center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2

zoom_level = get_zoom_level([lon_min, lat_min, lon_max, lat_max], map_width, map_height)

nrows, ncols = np_array.shape
lons = np.linspace(extent[0], extent[1], ncols)
lats = np.linspace(extent[3], extent[2], nrows)

lon_grid, lat_grid = np.meshgrid(lons, lats)

flat_lons = lon_grid.flatten()
flat_lats = lat_grid.flatten()
flat_vals = np_array.flatten()

valid_mask = ~np.isnan(flat_vals)
flat_lons = flat_lons[valid_mask]
flat_lats = flat_lats[valid_mask]
flat_vals = flat_vals[valid_mask]

hover_texts = [f"Value: {v:.2f}" for v in flat_vals]

fig = go.Figure()

fig.add_trace(go.Scattermapbox(
    lat=flat_lats,
    lon=flat_lons,
    mode='markers',
    marker=dict(size=5, color='rgba(0,0,0,0)'),
    hoverinfo='text',
    hovertext=hover_texts,
    showlegend=False,
))

fig.update_layout(
    width=map_width,
    height=map_height,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style='open-street-map',
        center=dict(lat=center_lat, lon=center_lon),
        zoom=zoom_level * 1.0,
        layers=[layer]
    )
)

fig.add_traces(boundary_traces)

fig.show()
# png_bytes = fig.to_image(format="png")
# display(Image(png_bytes))

In [ ]:

np_array = geemap.ee_to_numpy(
    floodRiskCategories.unmask(-9999),
    region=city_roi,
    scale=scale,
)
np_array = np.squeeze(np_array)
np_array = np_array.astype(float)
np_array[np_array == -9999] = np.nan

min_val = 1
max_val = 5
array_scaled_for_cmap = (np_array - min_val) / (max_val - min_val)

colored = rainbow_cmap(array_scaled_for_cmap)
alpha_channel = np.where(np.isnan(np_array), 0, 255).astype(np.uint8)
rgb_array = (colored[:, :, :3] * 255).astype(np.uint8)
rgba_array = np.dstack((rgb_array, alpha_channel))
image = PILImage.fromarray(rgba_array)
buffer = io.BytesIO()
image.save(buffer, format='PNG')
buffer.seek(0)
encoded_image = "data:image/png;base64," + base64.b64encode(buffer.read()).decode()

lon_min, lon_max, lat_min, lat_max = extent

image_coordinates = [
    [lon_min, lat_max],
    [lon_max, lat_max],
    [lon_max, lat_min],
    [lon_min, lat_min],
]

layer = dict(
    sourcetype='image',
    source=encoded_image,
    coordinates=image_coordinates,
    opacity=1
)

center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2

zoom_level = get_zoom_level([lon_min, lat_min, lon_max, lat_max], map_width, map_height)

nrows, ncols = np_array.shape
lons = np.linspace(extent[0], extent[1], ncols)
lats = np.linspace(extent[3], extent[2], nrows)

lon_grid, lat_grid = np.meshgrid(lons, lats)

flat_lons = lon_grid.flatten()
flat_lats = lat_grid.flatten()
flat_vals = np_array.flatten()

valid_mask = ~np.isnan(flat_vals)
flat_lons = flat_lons[valid_mask]
flat_lats = flat_lats[valid_mask]
flat_vals = flat_vals[valid_mask]

hover_texts = [f"Value: {v:.2f}" for v in flat_vals]

fig = go.Figure()

fig.add_trace(go.Scattermapbox(
    lat=flat_lats,
    lon=flat_lons,
    mode='markers',
    marker=dict(size=5, color='rgba(0,0,0,0)'),
    hoverinfo='text',
    hovertext=hover_texts,
    showlegend=False,
))

fig.update_layout(
    width=map_width,
    height=map_height,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style='open-street-map',
        center=dict(lat=center_lat, lon=center_lon),
        zoom=zoom_level * 1.0,
        layers=[layer]
    )
)

fig.add_traces(boundary_traces)

fig.show()
# png_bytes = fig.to_image(format="png")
# display(Image(png_bytes))